In [0]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import DoubleType

sales_order_detail_path = "/mnt/businesscase/silver/SalesOrderDetailParquet"
product_path = "/mnt/businesscase/silver/ProductParquet"

# Read the Parquet  table
product_df = spark.read.parquet(product_path)
sales_order_detail_df=spark.read.parquet(sales_order_detail_path)


In [0]:

# Perform the join using 'ProductID' as the common key
joined_df = sales_order_detail_df.join(product_df, 'ProductID', 'inner') \
    .withColumn('TotalListPrice', col('OrderQty') * col('ListPrice')) \
    .withColumn('TotalCost', col('OrderQty') * col('StandardCost'))

selected_columns = sales_order_detail_df.columns + ['TotalListPrice', 'TotalCost']
sales_order_detail_df = joined_df.select(selected_columns)  



In [0]:
from pyspark.sql.functions import col

# Calculate and add the 'TotalDiscount' column
sales_order_detail_df = sales_order_detail_df.withColumn('TotalDiscount', (col('TotalListPrice') * col('UnitPriceDiscount')))

# Calculate and add the 'NewLineTotal' column
sales_order_detail_df = sales_order_detail_df.withColumn('NewLineTotal', (col('TotalListPrice') - col('TotalDiscount')))

# Calculate and add the 'TotalProfit' column
sales_order_detail_df = sales_order_detail_df.withColumn('TotalProfit', (col('NewLineTotal') - col('TotalCost')))


In [0]:
# Select only the new columns you want to display
#selected_new_columns = ['TotalListPrice', 'TotalCost', 'TotalDiscount', 'NewLineTotal', 'TotalProfit']
#new_columns_df = sales_order_detail_df.select(selected_new_columns)

# Display the DataFrame with the selected new columns using the 'display' function
#display(new_columns_df)


In [0]:
# Save the DataFrame as a Parquet file in the "gold" folder
path="/mnt/businesscase/gold/SalesOrderDetailParquet"
sales_order_detail_df = sales_order_detail_df.coalesce(1)
sales_order_detail_df = sales_order_detail_df.repartition(1)
sales_order_detail_df.coalesce(1).write.parquet(path, mode="overwrite")


In [0]:
#Move data to silver folder as parquet format

parquet_list=["CustomerParquet","AddressParquet","ProductParquet"]

for address in parquet_list:
    path = "/mnt/businesscase/silver/"
    # Read the Delta files into a DataFrame
    path+=address
    # Read the Delta table
    product_df = spark.read.parquet(path)
    path=path.replace('silver','gold')
    
    # Save the DataFrame as a Parquet file in the "gold" folder
    product_df.coalesce(1).write.parquet(path, mode="overwrite")